<a href="https://colab.research.google.com/github/rsethi21/na_for_systems_bio/blob/update_train/manually_scale_kds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/rsethi21/na_for_systems_bio.git

Cloning into 'na_for_systems_bio'...
remote: Enumerating objects: 1190, done.
remote: Counting objects: 100% (78/78), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 1190 (delta 34), reused 38 (delta 20), pack-reused 1112
Receiving objects: 100% (1190/1190), 43.47 MiB | 7.39 MiB/s, done.
Resolving deltas: 100% (678/678), done.


In [2]:
!pip3 install -r na_for_systems_bio/requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 745.3 kB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider
import argparse
import pdb
import json
import os
import sys
sys.path.append(f"{sys.path[0]}/na_for_systems_bio")

from src.substrate import Substrate
from src.interaction import Interaction
from src.rate import Rate
from src.network import Network
from src.parse import parse_rates, parse_substrates, parse_interactions
from src.score import error

In [78]:
rates = parse_rates("na_for_systems_bio/input/input_expanded/rates.csv")
substrates = parse_substrates("na_for_systems_bio/input/input_expanded/substrates.csv", rates)
interactions = parse_interactions("na_for_systems_bio/input/input_expanded/interactions.csv", rates, substrates)
network = Network("example", rates, interactions, substrates)

In [59]:
time = np.array([i for i in range(1001)])
derivatives = network.get_representation_dydt()
for l, t in derivatives.items():
    print(f"{l} = {t}")
print()

d[AKT]/dt = k3[pAKT] - r3[AKT][(PIP3^n2 + Km2^n2)/Km2^n2]
d[pAKT]/dt = r3[AKT][(PIP3^n2 + Km2^n2)/Km2^n2] - k3[pAKT]
d[PTEN]/dt = k4[pPTEN] - r4[PTEN][GSK3B] + s2[LPS][pPTEN]
d[pPTEN]/dt = r4[PTEN][GSK3B] - k4[pPTEN] - s2[LPS][pPTEN]
d[PIP2]/dt = k2[PIP3] - r2[PIP2][PI3K][Km1^n1/(PTEN^n1 + Km1^n1)]
d[PIP3]/dt = r2[PIP2][PI3K][Km1^n1/(PTEN^n1 + Km1^n1)] - k2[PIP3]
d[PI3K]/dt = k1[PI3Ks] - r1[PI3K] + s3[Gio][PI3Ks] - s4[PI3K][LY294-002]
d[PI3Ks]/dt = r1[PI3K] - k1[PI3Ks] - s3[PI3Ks][Gio] + s4[LY294-002][PI3K] + k11[Ca2+]
d[GSK3B]/dt = k5[pGSK3B] - r5[GSK3B][pAKT] + s1[HDACi] - s5[shRNA_GSK3B][GSK3B]
d[pGSK3B]/dt = r5[GSK3B][pAKT] - k5[pGSK3B]
d[TNFa]/dt = k6[PTEN] - r6[TNFa][pAKT]
d[LPS]/dt = 0
d[HDACi]/dt = 0
d[Phagocytosis]/dt = k7[pAKT] - r7[Phagocytosis]
d[LY294-002]/dt = 0
d[ATP]/dt = -r10[ATP]
d[P2Y12act]/dt = k8[P2Y12s][ADP] - r8[P2Y12act] + k10[Ca2+][ADP]
d[P2Y12s]/dt = r8[P2Y12act] - k8[P2Y12s][ADP] - k10[Ca2+][ADP]
d[Gio]/dt = k9[P2Y12act] - r9[Gio][Gio]
d[Ca2+]/dt = -r12[Ca2+]

In [89]:
with open("na_for_systems_bio/input/input_expanded_2/fitted_params_scaled.json", "r") as fitted_params:
    parameters = json.load(fitted_params)
network.set_parameters(list(parameters.values()), list(parameters.keys()))
network.initial_parameter_values = parameters

In [61]:
parameters_to_scale = network.find_rate_pairs()
for p_id, p in network.parameters.items():
  if not p.fixed:
    if p_id not in "".join(list(parameters_to_scale.keys())):
      parameters_to_scale[p_id] = p

In [76]:
def plot(**updated_parameters):
  print()
  time = np.array([i for i in range(1001)])
  updates = {}
  for k, v in updated_parameters.items():
    updates[k] = 10**v
  network.apply_scaling(list(updates.values()), list(updated_parameters.keys()))
  mu, fig = network.graph_distributions(time, 10, substrates_to_plot=["LPS", "pAKT", "Phagocytosis", "PTEN"], normalize=True, output_figure=True)
  plt.figure(fig)
  plt.legend(loc="lower right")
  print({p: v.value for p, v in network.parameters.items()})
  plt.show()

In [90]:
{p:v.value for p, v in network.parameters.items()}

{'k1': 0.02665745450117281,
 'k2': 0.061896058485546385,
 'k3': 0.08989766450921034,
 'k4': 0.005149693931201939,
 'k5': 0.004808427056403533,
 'k6': 1.0,
 'k7': 0.008619086553313844,
 'k8': 0.336,
 'k9': 6.0,
 'k10': 0.144,
 'k11': 0.01,
 'r1': 0.024129922406274797,
 'r2': 0.017184890082343595,
 'r3': 0.0028035097113273243,
 'r4': 0.008994262309614622,
 'r5': 0.0008904108460071361,
 'r6': 1.0,
 'r7': 0.0870005936946528,
 'r8': 1.2,
 'r9': 3.0,
 'r10': 0.1,
 'r11': 0.48,
 'r12': 0.06,
 's1': 0.008981261572474834,
 's2': 0.0021450424947525395,
 's3': 0.0006,
 's4': 0.06946921913641102,
 's5': 0.0074781546276185915,
 'Km1': 0.6215631534357107,
 'Km2': 0.4088917703242811,
 'n1': 1.0,
 'n2': 2.0}

In [87]:
network.set_parameters(list(parameters.values()), list(parameters.values()))

In [82]:
def reset(network):
  network.set_parameters(list(network.initial_parameter_values.values()), list(network.initial_parameter_values.values()))

- only apply scale when asked and to the original parameters
- be able to reset to original

In [83]:
reset(network)
sliders = {}
for p, v in parameters_to_scale.items():
  if type(v) == list:
    val = np.log10(v[0].value/network.initial_parameter_values[v[0].identifier])
    sliders[p] = FloatSlider(value = 0, min=-10, max=10, step=1, description=f"{p}:")
  else:
    val = np.log10(v.value/network.initial_parameter_values[v.identifier])
    sliders[p] = FloatSlider(value = 0, min=-10, max=10, step=1, description=f"{p}:")
interact(plot, **sliders)

interactive(children=(FloatSlider(value=0.0, description='k3,r3:', max=10.0, min=-10.0, step=1.0), FloatSlider…

<function __main__.plot(**updated_parameters)>